# 6.3 継承

6.1, 6.2で説明したクラスを使えば、オブジェクトを作ることが出来る。さらにRubyのクラスを拡張し、別なクラスを作る継承と呼ばれる概念がある。オブジェクト指向では、拡張元のクラスを**スーパークラス**(親クラス)、拡張先のクラスを**サブクラス**(子クラス)と呼ぶ。Rubyで継承をする場合は、```class 子 < 親```と記述する。継承を行うと、親クラスのすべてのpublicフィールド・メソッド(public, privateに関しては、6.1参照)にアクセスすることが出来る。下記の例では、スーパークラス*Person*の*@name*のgetterメソッドは、サブクラス*Programmer*では、再定義をする必要がない(**再利用性**)。一方コンストラクタ・*greet*メソッドは、再定義がされている。サブクラスで親クラスのメソッドを再定義(上書き)することを**メソッドのオーバーライド**と呼ぶ。オーバライド先のメソッドでは、**super**キーワードを使うことでスーパークラスのメソッドを呼び出すことが出来る。

In [1]:
class Person
    attr_reader :name
  
    def initialize(name)
        @name = name
    end
    
    def greet
        "I am #{@name}."
    end
end

:greet

In [2]:
class Programmer < Person # Programmer(サブクラス) < Person(スーパークラス)
    attr_reader :language
    
    def initialize(name, language) # コンストラクタのオーバーライド
      super(name) # スーパークラスのコンストラクタを呼び出している。(@nameフィールドの初期化)
      @language = language
    end
    
  def greet # greetのオーバーライド
      "#{super} I use #{@language}." # スーパークラスのgreetを呼び出している。
  end
end

:greet

In [3]:
john = Person.new("John")

#<Person:0x005588e6658640 @name="John">

In [4]:
mike = Programmer.new("Mike", "Ruby")

#<Programmer:0x005588e6593340 @name="Mike", @language="Ruby">

## 多態性(Polymorphism)

ある２つのクラスが継承関係にあるとき、この関係を**IS-Aの関係**と呼ぶ。つまり、```子 is a 親```が成り立つ(逆は成り立たない)。この関係が成り立つとき、すべてのインスタンスに対して、親クラスの振る舞い(メソッド呼び出し)が保証される。下記の例では、*Person*クラスと*Programmer*クラスのインスタンスの配列すべてに対して、Personクラスの振る舞い(*greet*・*name*メソッド)が保証される。

IS-Aの関係が成り立つので、下記の配列は、Personの配列(persons)であるとみなせる。

In [5]:
persons = [Person.new("John"), Programmer.new("Mike", "Ruby")]

[#<Person:0x005588e653be38 @name="John">, #<Programmer:0x005588e653bdc0 @name="Mike", @language="Ruby">]

In [6]:
persons.map{|person| person.greet}

["I am John.", "I am Mike. I use Ruby."]

In [7]:
persons.map{|person| person.name}

["John", "Mike"]

*Person is a Programer*は、成り立たないので、languageメソッドは保証されない。

In [8]:
persons.map{|person| person.language}

NoMethodError: undefined method `language' for #<Person:0x005588e653be38 @name="John">

上記の例では、greetメソッドを呼び出している。このとき、インスタンス(クラス)によって振る舞いが異なる。このように、実行される処理の実体が、コールされたメッセージ(メソッド)ではなく、メッセージを受けたオブジェクト(レシーバ)によって異なる性質のことを**多態性(Polymorphism、 多相性、多型性とも)**と呼ぶ。

## ダック・タイピング
前節では、継承による多層性を説明した。オブジェクト指向(を利用した言語)では、他にも様々な仕組みを利用して多層性を利用することができる。例えばコード上で厳密に型を定義し、コンパイラでチェックするような(静的型付け)言語では、IS-A関係が成り立つような型にしなければ、配列の型はルートクラス(Objectクラス)となってしまう。

```java
List<Person> person = Arrays.asList(new Person("John"), new Programmer("Mike", "C"));
List<Object> foo = Arrays.asList(new Person("John"), new Programmer("Mike", "C"), new Dog("Toby"));
```

結果、使いたいメソッドの保証がされなくなってしまう(もちろんJavaを始め、オブジェクト指向をサポートする言語では、別な回避策がある)。Rubyでは、動的型付けのため、型よりも振る舞いを重視する。つまり、**同じインタフェース(メソッド)を実装するオブジェクト同士が、それぞれがどのような継承階層を持っているのかということと無関係に、相互に交換可能である**。このような性質を**ダック・タイピング**と呼ぶ。この名前は、[ダックテスト](https://ja.wikipedia.org/wiki/ダック・テスト)に由来する。

```
"If it walks like a duck and quacks like a duck, it must be a duck"
（もしもそれがアヒルのように歩き、アヒルのように鳴くのなら、それはアヒルである）
```

以下に、ダック・タイピングを利用した多態性の例を紹介する。

In [9]:
class Dog # Dog is not a Person.
  def initialize(name)
    @name = name
  end
  
  def greet # but Dog can greet.
    "bow-wow!"
  end
end

:greet

Dogを含めた配列は、personsではなく、振る舞いを重視し、挨拶集団(greeter)とするべきである。

In [10]:
# persons.push(Dog.new("Toby")) Dogは、Person or Programmer?

In [11]:
greeters = [Person.new("John"), Programmer.new("Mike", "Ruby"), Dog.new("Toby")]

[#<Person:0x005588e63a3af8 @name="John">, #<Programmer:0x005588e63a3a30 @name="Mike", @language="Ruby">, #<Dog:0x005588e63a3968 @name="Toby">]

In [12]:
greeters.map{|greeter| greeter.greet} # 挨拶が可能なら挨拶者!

["I am John.", "I am Mike. I use Ruby.", "bow-wow!"]

## 演習

Immutableなコレクションクラス、ImQueueを実装せよ。Queueについては、ImStackや[wiki](https://ja.wikipedia.org/wiki/キュー_(コンピュータ)等を参考にせよ。今回メソッドは、push(enqueue), pop(dequeue)のみを実装すれば良い。

In [13]:
class ImStack
  attr_accessor :stack
  def initialize(stack=[])
    @stack = stack.freeze
  end
    
  def push(v)
    ImStack.new(@stack.dup.concat([v]))
  end
    
  def pop
    ImStack.new(@stack[0..-2])
  end
end

:pop

In [14]:
stack = (0..4).reduce(ImStack.new){|stack, n| stack.push(n)}

#<ImStack:0x005588e6340570 @stack=[0, 1, 2, 3, 4]>

In [15]:
queue = (0..4).reduce(ImQueue.new){|queue, n| queue.push(n)}

NameError: uninitialized constant ImQueue

In [16]:
stack.pop.pop

#<ImStack:0x005588e60df470 @stack=[0, 1, 2]>

In [17]:
queue.pop.pop

NoMethodError: undefined method `pop' for nil:NilClass

さらに、ダックタイピングを利用して、下記のコードを簡潔に書き直せ。

In [18]:
(0..4).reduce([ImStack.new, ImQueue.new]){|sq, n| [sq[0].push(n), sq[1].push(n)]}

NameError: uninitialized constant ImQueue

## 継承よりコンポジションを選ぶ

継承はコードを再利用をするための強力な機能である。しかし、常に再利用性のための最善の手段ではない。継承の代替案として、コンポジションがある。継承がIS-Aの関係であることに対して、コンポジションは**HAS-Aの関係**(6.1)を指す。本章の例では、*ImStack*・*ImQueue*が*Array*を持っている(隠蔽している)。例えば*Programmer*クラスは、コンポジションを利用することで、以下のように書き換えることが出来る。

*Programmer* 再掲

```ruby
class Programmer < Person 
    attr_reader :language
    
    def initialize(name, language) 
      super(name)
      @language = language
    end
    
  def greet 
      "#{super} I use #{@language}." 
  end
end
```

In [19]:
class CompProgrammer
  attr_reader :language
    
  def initialize(name, language)
    @person = Person.new(name) # Personを持つ。
    @language = language
  end
  
  def name # 他のオブジェクトのメソッドを呼び出し、任せることを委譲(delegation)と呼ぶ。
    @person.name
  end
  
  def greet
    "#{@person.greet} I use #{@language}."
  end
end

:greet

In [20]:
CompProgrammer.new("Mike", "Ruby").greet

"I am Mike. I use Ruby."

コンポジションに比べ、**継承はカプセル化を破る**。言い換えれば、サブクラスは適切に機能するために、スーパークラスの実装に依存する。具体例のために、以下に集合を表すクラス*MySet*と要素の挿入回数をカウントするクラス*CountedSet*を実装する。

In [21]:
class MySet
  attr_reader :array
  
  def initialize
    @array = []
  end
  
  def add(v)
    unless @array.include?(v)
      @array.push(v)
    end
  end
  
  def addAll(values)
    values.each{|v| add(v)}
  end
end

:addAll

In [22]:
set = MySet.new

#<MySet:0x005588e5f12228 @array=[]>

In [23]:
set.addAll([1, 1, 2, 3, 3, 4])

[1, 1, 2, 3, 3, 4]

In [24]:
set.array

[1, 2, 3, 4]

In [25]:
class CountedSet < MySet
  attr_reader :count
  
  def initialize
    super
    @count = 0
  end
  
  def add(v)
    super(v)
    @count += 1
  end
  
  def addAll(values)
    super(values)
    @count += values.length
  end
end

:addAll

一見すると正常に動作しているかのように見える。

In [26]:
cset = CountedSet.new

#<CountedSet:0x005588e5ce8150 @array=[], @count=0>

In [27]:
cset.add(1)

1

In [28]:
cset.add(2)

2

In [29]:
cset.add(1)

3

In [30]:
cset.add(3)

4

In [31]:
cset.count

4

しかし、*addAll*メソッドでは、予期しない結果となってしまう。これは、addAllメソッドが、内部的に*add*メソッドを呼び出しているためであり。これが、スーパークラスの実装に依存してしまっている状態である。例えば、今回はaddAllメソッドをオーバーライドしないことで、この問題は回避することができる。しかし、MySetクラスのaddAllメソッドの実装がいつ変更されるかは、保証出来ないため、とても危険である。

In [32]:
cset.addAll([4, 5, 6])

10

MySetの例を通して継承の危険性を説明した。継承を実行できる条件は、スーパークラスの変更を追従可能な場合である。加えて、IS-Aが必ず成り立つかどうかも、大きなポイントなので抑えておきたい。Javaのプラットフォームライブラリには、この原則を破っているクラスがいくつか存在している。例えば、Stackは、Vectorを継承するべきではない(Stack is not a Vector)。

## 演習
CountedSetのソースコードをコンポジションを利用して、変更を加え、問題を解決せよ。

## チェックポイント

- 継承による利点は何か
- ポリモーフィズム・ダックタイピングとは何か
- 継承による注意点は何か